# 09 - MLflow Básico: Tracking y Experiment Management

## 🎯 Objetivos
- Aprender los fundamentos de MLflow
- Tracking de experimentos y métricas
- Logging de parámetros, métricas y artefactos
- Comparación de modelos
- Uso de MLflow UI

## 📚 Tecnologías
- **MLflow**: Experiment tracking, model registry
- **Scikit-learn**: Modelos básicos
- **Pandas**: Manipulación de datos

## ⭐ Complejidad: Básico

## 1. Instalación y Setup

In [ ]:
# Instalar dependencias
!pip install mlflow scikit-learn pandas numpy matplotlib seaborn -q

In [ ]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

print(f"✅ MLflow version: {mlflow.__version__}")

## 2. Configuración de MLflow

MLflow guarda los experimentos en un directorio local por defecto.

In [ ]:
# Configurar tracking URI (local)
mlflow.set_tracking_uri("./mlruns")

# Crear o establecer experimento
experiment_name = "mlflow_basico_tutorial"
mlflow.set_experiment(experiment_name)

print(f"📊 Experimento: {experiment_name}")
print(f"📁 Tracking URI: {mlflow.get_tracking_uri()}")

## 3. Crear Dataset de Ejemplo

Generamos un dataset sintético de clasificación binaria.

In [ ]:
# Generar datos sintéticos
X, y = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    n_classes=2,
    random_state=42
)

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"📊 Train set: {X_train.shape}")
print(f"📊 Test set: {X_test.shape}")
print(f"📊 Class distribution: {np.bincount(y)}")

## 4. Ejemplo Básico: Un Solo Experimento

Veamos cómo trackear un experimento simple.

In [ ]:
# Iniciar un run de MLflow
with mlflow.start_run(run_name="logistic_regression_basic"):
    
    # 1. Log de parámetros
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_param("max_iter", 100)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("test_size", 0.2)
    
    # 2. Entrenar modelo
    model = LogisticRegression(max_iter=100, random_state=42)
    model.fit(X_train, y_train)
    
    # 3. Predicciones
    y_pred = model.predict(X_test)
    
    # 4. Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # 5. Log de métricas
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    
    # 6. Log del modelo
    mlflow.sklearn.log_model(model, "model")
    
    # 7. Crear y guardar un artefacto (gráfico)
    plt.figure(figsize=(8, 6))
    plt.scatter(range(len(y_test)), y_test, alpha=0.5, label='Real')
    plt.scatter(range(len(y_pred)), y_pred, alpha=0.5, label='Predicción')
    plt.xlabel('Muestras')
    plt.ylabel('Clase')
    plt.title('Predicciones vs Real')
    plt.legend()
    plt.savefig('predictions.png')
    mlflow.log_artifact('predictions.png')
    plt.close()
    
    print(f"✅ Experimento completado!")
    print(f"📊 Accuracy: {accuracy:.4f}")
    print(f"📊 Precision: {precision:.4f}")
    print(f"📊 Recall: {recall:.4f}")
    print(f"📊 F1-Score: {f1:.4f}")

## 5. Comparación de Múltiples Modelos

Entrenemos varios modelos y comparemos sus resultados.

In [ ]:
# Definir modelos a comparar
models = {
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "GradientBoosting": GradientBoostingClassifier(n_estimators=100, random_state=42)
}

results = []

# Entrenar y trackear cada modelo
for model_name, model in models.items():
    with mlflow.start_run(run_name=f"{model_name}_comparison"):
        
        # Log parámetros
        mlflow.log_param("model_type", model_name)
        mlflow.log_param("dataset_size", len(X_train))
        
        # Entrenar
        model.fit(X_train, y_train)
        
        # Predecir
        y_pred = model.predict(X_test)
        
        # Métricas
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # Log métricas
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)
        
        # Log modelo
        mlflow.sklearn.log_model(model, "model")
        
        # Guardar resultados
        results.append({
            "Model": model_name,
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1-Score": f1
        })
        
        print(f"✅ {model_name} - Accuracy: {accuracy:.4f}, F1: {f1:.4f}")

# Mostrar tabla comparativa
results_df = pd.DataFrame(results)
print("\n📊 Comparación de Modelos:")
print(results_df.to_string(index=False))

## 6. Logging de Artefactos Complejos

Guardemos visualizaciones y archivos adicionales.

In [ ]:
with mlflow.start_run(run_name="artifacts_example"):
    
    # Entrenar modelo
    model = RandomForestClassifier(n_estimators=50, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Métricas
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)
    
    # 1. Guardar importancia de features
    feature_importance = pd.DataFrame({
        'feature': [f'feature_{i}' for i in range(X_train.shape[1])],
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # Gráfico de importancia
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importance['feature'][:10], feature_importance['importance'][:10])
    plt.xlabel('Importance')
    plt.title('Top 10 Feature Importance')
    plt.tight_layout()
    plt.savefig('feature_importance.png')
    mlflow.log_artifact('feature_importance.png')
    plt.close()
    
    # 2. Guardar CSV con importancia
    feature_importance.to_csv('feature_importance.csv', index=False)
    mlflow.log_artifact('feature_importance.csv')
    
    # 3. Guardar matriz de confusión
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('Real')
    plt.xlabel('Predicción')
    plt.savefig('confusion_matrix.png')
    mlflow.log_artifact('confusion_matrix.png')
    plt.close()
    
    # 4. Guardar metadata como JSON
    import json
    metadata = {
        "model": "RandomForestClassifier",
        "n_estimators": 50,
        "train_size": len(X_train),
        "test_size": len(X_test),
        "accuracy": float(accuracy),
        "features": X_train.shape[1]
    }
    
    with open('metadata.json', 'w') as f:
        json.dump(metadata, f, indent=2)
    mlflow.log_artifact('metadata.json')
    
    print("✅ Todos los artefactos guardados!")
    print(f"📊 Top 5 features más importantes:")
    print(feature_importance.head())

## 7. Uso de Tags y Búsqueda de Experimentos

In [ ]:
# Experimento con tags
with mlflow.start_run(run_name="tagged_experiment") as run:
    
    # Agregar tags
    mlflow.set_tag("team", "data-science")
    mlflow.set_tag("project", "tutorial-basico")
    mlflow.set_tag("environment", "development")
    mlflow.set_tag("version", "1.0")
    
    # Entrenar modelo simple
    model = LogisticRegression(random_state=42)
    model.fit(X_train, y_train)
    
    # Log métricas
    accuracy = accuracy_score(y_test, model.predict(X_test))
    mlflow.log_metric("accuracy", accuracy)
    
    # Guardar modelo
    mlflow.sklearn.log_model(model, "model")
    
    print(f"✅ Run ID: {run.info.run_id}")
    print(f"📊 Accuracy: {accuracy:.4f}")

## 8. Búsqueda y Recuperación de Experimentos

In [ ]:
# Buscar todos los runs del experimento actual
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment = client.get_experiment_by_name(experiment_name)

# Obtener todos los runs
runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["metrics.accuracy DESC"],
    max_results=10
)

print(f"📊 Total de runs encontrados: {len(runs)}\n")

# Mostrar información de los mejores runs
for i, run in enumerate(runs[:5], 1):
    print(f"\n{i}. Run: {run.info.run_name}")
    print(f"   Run ID: {run.info.run_id}")
    print(f"   Status: {run.info.status}")
    
    # Parámetros
    if run.data.params:
        print(f"   Parámetros:")
        for key, value in run.data.params.items():
            print(f"      - {key}: {value}")
    
    # Métricas
    if run.data.metrics:
        print(f"   Métricas:")
        for key, value in run.data.metrics.items():
            print(f"      - {key}: {value:.4f}")

## 9. Cargar y Usar un Modelo Guardado

In [ ]:
# Obtener el mejor run (por accuracy)
best_run = runs[0]
best_run_id = best_run.info.run_id

print(f"🏆 Mejor modelo:")
print(f"   Run ID: {best_run_id}")
print(f"   Run Name: {best_run.info.run_name}")
print(f"   Accuracy: {best_run.data.metrics.get('accuracy', 0):.4f}")

# Cargar el modelo
model_uri = f"runs:/{best_run_id}/model"
loaded_model = mlflow.sklearn.load_model(model_uri)

print(f"\n✅ Modelo cargado exitosamente!")
print(f"📊 Tipo de modelo: {type(loaded_model).__name__}")

# Hacer predicciones con el modelo cargado
predictions = loaded_model.predict(X_test[:5])
print(f"\n🔮 Predicciones de ejemplo: {predictions}")
print(f"📌 Valores reales: {y_test[:5]}")

## 10. Registro de Modelos (Model Registry)

MLflow permite registrar modelos para producción.

In [ ]:
# Registrar el mejor modelo
model_name = "tutorial_best_classifier"

try:
    # Registrar modelo desde el run
    model_uri = f"runs:/{best_run_id}/model"
    registered_model = mlflow.register_model(model_uri, model_name)
    
    print(f"✅ Modelo registrado!")
    print(f"📦 Nombre: {registered_model.name}")
    print(f"📌 Versión: {registered_model.version}")
    
except Exception as e:
    print(f"⚠️ Nota: {e}")
    print(f"💡 El registro de modelos requiere un backend remoto (MySQL, PostgreSQL, etc.)")
    print(f"💡 Para uso local, puedes cargar modelos directamente con el run_id")

## 11. Visualización con MLflow UI

Para ver todos tus experimentos en una interfaz web, ejecuta en la terminal:

```bash
mlflow ui --port 5000
```

Luego abre en tu navegador: http://localhost:5000

### Características de MLflow UI:
- 📊 Comparación visual de experimentos
- 📈 Gráficos de métricas
- 🔍 Búsqueda y filtrado avanzado
- 📁 Navegación de artefactos
- 🏷️ Gestión de tags
- 📦 Registro de modelos

## 12. Resumen y Mejores Prácticas

### ✅ Conceptos Clave:
1. **Experiments**: Agrupan runs relacionados
2. **Runs**: Una ejecución individual de código ML
3. **Parameters**: Configuración del modelo (hiperparámetros)
4. **Metrics**: Resultados cuantitativos (accuracy, loss, etc.)
5. **Artifacts**: Archivos generados (modelos, gráficos, datos)
6. **Tags**: Metadata adicional para organización

### 💡 Mejores Prácticas:
- ✅ Usa nombres descriptivos para runs y experimentos
- ✅ Loggea todos los parámetros relevantes
- ✅ Registra múltiples métricas para análisis completo
- ✅ Guarda artefactos importantes (modelos, gráficos)
- ✅ Usa tags para organizar experimentos por proyecto/equipo
- ✅ Compara múltiples modelos en el mismo experimento
- ✅ Documenta tus experimentos con tags y descripciones

### 🚀 Próximos Pasos:
- Integrar con frameworks de deep learning (PyTorch, TensorFlow)
- Configurar backend remoto para Model Registry
- Automatizar experimentos con Airflow
- Deployar modelos con MLflow Models

In [ ]:
print("🎉 Tutorial de MLflow Básico completado!")
print(f"\n📊 Estadísticas finales:")
print(f"   - Experimento: {experiment_name}")
print(f"   - Total de runs: {len(runs)}")
print(f"   - Mejor accuracy: {runs[0].data.metrics.get('accuracy', 0):.4f}")
print(f"\n💻 Para ver la UI ejecuta: mlflow ui --port 5000")